## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ostrovnoy,RU,68.0531,39.5131,14.14,86,96,22.17,overcast clouds
1,1,Thompson,CA,55.7435,-97.8558,-11.04,64,20,4.61,few clouds
2,2,Grand Gaube,MU,-20.0064,57.6608,79.38,80,84,3.00,broken clouds
3,3,Alofi,NU,-19.0595,-169.9187,75.09,94,100,4.61,overcast clouds
4,4,Luderitz,NaN,-26.6481,15.1594,68.59,75,16,4.94,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Grand Gaube,MU,-20.0064,57.6608,79.38,80,84,3.00,broken clouds
3,3,Alofi,NU,-19.0595,-169.9187,75.09,94,100,4.61,overcast clouds
6,6,Castro,BR,-24.7911,-50.0119,77.77,66,68,7.36,broken clouds
11,11,Rikitea,PF,-23.1203,-134.9692,79.02,71,66,16.06,broken clouds
13,13,Port Alfred,ZA,-33.5906,26.8910,75.65,81,0,27.47,clear sky
17,17,Georgetown,MY,5.4112,100.3354,78.73,93,40,0.00,scattered clouds
20,20,Yenagoa,NG,4.9247,6.2642,83.79,59,3,10.02,clear sky
21,21,Thinadhoo,MV,0.5333,72.9333,82.72,70,33,12.17,scattered clouds
23,23,Tamandare,BR,-8.7597,-35.1047,84.85,59,96,11.79,overcast clouds
27,27,Abong Mbang,CM,3.9833,13.1833,75.02,27,55,2.75,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
len(preferred_cities_clean_df)

195

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", 
                                      "Country", 
                                      "Max Temp", 
                                      "Current Description", 
                                      "Lat", 
                                      "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Grand Gaube,MU,79.38,broken clouds,-20.0064,57.6608,
3,Alofi,NU,75.09,overcast clouds,-19.0595,-169.9187,
6,Castro,BR,77.77,broken clouds,-24.7911,-50.0119,
11,Rikitea,PF,79.02,broken clouds,-23.1203,-134.9692,
13,Port Alfred,ZA,75.65,clear sky,-33.5906,26.8910,
17,Georgetown,MY,78.73,scattered clouds,5.4112,100.3354,
20,Yenagoa,NG,83.79,clear sky,4.9247,6.2642,
21,Thinadhoo,MV,82.72,scattered clouds,0.5333,72.9333,
23,Tamandare,BR,84.85,overcast clouds,-8.7597,-35.1047,
27,Abong Mbang,CM,75.02,broken clouds,3.9833,13.1833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
   
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
            
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Grand Gaube,MU,79.38,broken clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
3,Alofi,NU,75.09,overcast clouds,-19.0595,-169.9187,Taloa Heights
6,Castro,BR,77.77,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
11,Rikitea,PF,79.02,broken clouds,-23.1203,-134.9692,People ThankYou
13,Port Alfred,ZA,75.65,clear sky,-33.5906,26.8910,The Halyards Hotel
17,Georgetown,MY,78.73,scattered clouds,5.4112,100.3354,Cititel Penang
20,Yenagoa,NG,83.79,clear sky,4.9247,6.2642,Aridolf Resort Wellness and Spa
23,Tamandare,BR,84.85,overcast clouds,-8.7597,-35.1047,Pousada 100 Passos
27,Abong Mbang,CM,75.02,broken clouds,3.9833,13.1833,Auberge Angle Sur
28,Doume,CM,76.03,broken clouds,4.2333,13.4500,Hotel Universel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))